In [1]:
pip install google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: anyio
    Found existing installation: anyio 4.2.0
    Uninstalling anyio-4.2.0:
      Successfully uninstalled anyio-4.2.0
Note: you may need to restart the kernel to use updated packages.


## - Prompt 1 Gemini - flash

In [3]:
import os
from google import genai

# =====================
# CONFIGURAÇÃO
# =====================
API_KEY = os.getenv("", "")
MODEL_NAME = "gemini-2.5-flash"
IMAGE_PATH = "coloque o diretório da imagem aqui"

# =====================
# PROMPT
# =====================
PROMPT = (
    "Tarefa: Converta o Diagrama Entidade-Relacionamento (ER) em um esquema lógico relacional representado em formato JSON.\n\n"
    "Instruções:\n"
    "- \"attributes\": lista com todos os atributos da tabela\n"
    "- \"pk\": chave primária como lista (mesmo que contenha apenas um atributo)\n"
    "- \"fk\": lista de chaves estrangeiras (se existirem)\n\n"
    "Caso não existam chaves estrangeiras, omita o campo \"fk\".\n"
    "Use nomes significativos, mantendo consistência com o diagrama original.\n\n"
    "O modelo de saída deve seguir rigorosamente este formato:\n"
    "{\n"
    "  \"Entities\": {\n"
    "    \"EntityName\": {\n"
    "      \"attributes\": [\"x\", \"y\", \"z\"],\n"
    "      \"pk\": [\"a\"],\n"
    "      \"fk\": [\"b\", \"c\"]\n"
    "    }\n"
    "  }\n"
    "}"
)

# =====================
# EXECUÇÃO
# =====================
def main():
    client = genai.Client(api_key=API_KEY)

    with open(IMAGE_PATH, "rb") as f:
        image_bytes = f.read()

    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=[
            {"inline_data": {"mime_type": "image/png", "data": image_bytes}},
            {"text": PROMPT}
        ]
    )

    print("\n=== Saída do Gemini ===\n")
    print(response.text)

if __name__ == "__main__":
    main()



=== Saída do Gemini ===

```json
{
  "Entities": {
    "EMPLOYEE": {
      "attributes": ["Ssn", "Fname", "Minit", "Lname", "Bdate", "Address", "Salary", "Sex", "Dno", "Supervisor_Ssn", "Start_date"],
      "pk": ["Ssn"],
      "fk": ["Dno", "Supervisor_Ssn"]
    },
    "DEPARTMENT": {
      "attributes": ["Number", "Name", "Mgr_Ssn"],
      "pk": ["Number"],
      "fk": ["Mgr_Ssn"]
    },
    "DEPARTMENT_LOCATION": {
      "attributes": ["Dnumber", "Location"],
      "pk": ["Dnumber", "Location"],
      "fk": ["Dnumber"]
    },
    "PROJECT": {
      "attributes": ["Number", "Name", "Location", "Dno"],
      "pk": ["Number"],
      "fk": ["Dno"]
    },
    "DEPENDENT": {
      "attributes": ["Essn", "Name", "Sex", "Birth_date", "Relationship"],
      "pk": ["Essn", "Name"],
      "fk": ["Essn"]
    },
    "WORKS_ON": {
      "attributes": ["Essn", "Pno", "Hours"],
      "pk": ["Essn", "Pno"],
      "fk": ["Essn", "Pno"]
    }
  }
}
```


## Prompt 2 geminni 

In [5]:
import os
from google import genai

# =====================
# CONFIGURAÇÃO
# =====================
API_KEY = os.getenv("", "")
MODEL_NAME = "gemini-2.5-flash"
IMAGE_PATH = "coloque o diretório da imagem aqui"

# =====================
# PROMPT
# =====================
PROMPT = (
     "Tarefa: Converta o Diagrama Entidade-Relacionamento (ER) em um esquema lógico relacional representado em formato JSON.\n\n"
    "Instruções:\n"
    "Para cada entidade ou relacionamento, gere uma entrada com:\n\n"
    "- \"attributes\": lista com todos os atributos da tabela\n"
    "- \"pk\": chave primária como lista (mesmo que contenha apenas um atributo)\n"
    "- \"fk\": lista de chaves estrangeiras (se existirem)\n\n"
    "Caso não existam chaves estrangeiras, omita o campo \"fk\".\n"
    "Use nomes significativos, mantendo consistência com o diagrama original.\n\n"
    "Utilize as regras abaixo para decisão por tabela própria, adição de coluna ou fusão de tabelas:\n\n"
    "Relacionamentos 1:1\n"
    "1. (0,1) — (0,1)\n"
    "Tabela própria: ± (pode ser usada)\n"
    "Adição de coluna: ✔ (preferida)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "2. (0,1) — (1,1)\n"
    "Tabela própria: ✘ (não usar)\n"
    "Adição de coluna: ± (pode ser usada)\n"
    "Fusão de tabelas: ✔ (preferida)\n\n"
    "3. (1,1) — (1,1)\n"
    "Tabela própria: ✘ (não usar)\n"
    "Adição de coluna: ± (pode ser usada)\n"
    "Fusão de tabelas: ✔ (preferida)\n\n"
    "Relacionamentos 1:n\n"
    "1. (0,1) — (0,n)\n"
    "Tabela própria: ± (pode ser usada)\n"
    "Adição de coluna: ✔ (preferida)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "2. (0,1) — (1,n)\n"
    "Tabela própria: ± (pode ser usada)\n"
    "Adição de coluna: ✔ (preferida)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "3. (1,1) — (0,n)\n"
    "Tabela própria: ✘ (não usar)\n"
    "Adição de coluna: ✔ (preferida)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "4. (1,1) — (1,n)\n"
    "Tabela própria: ✘ (não usar)\n"
    "Adição de coluna: ✔ (preferida)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "Relacionamentos n:n\n"
    "1. (0,n) — (0,n)\n"
    "Tabela própria: ✔ (preferida)\n"
    "Adição de coluna: ✘ (não usar)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "2. (0,n) — (1,n)\n"
    "Tabela própria: ✔ (preferida)\n"
    "Adição de coluna: ✘ (não usar)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "3. (1,n) — (1,n)\n"
    "Tabela própria: ✔ (preferida)\n"
    "Adição de coluna: ✘ (não usar)\n"
    "Fusão de tabelas: ✘ (não usar)\n\n"
    "O modelo de saída deve seguir rigorosamente o seguinte formato:\n"
    "{\n"
    "  \"Entities\": {\n"
    "    \"EntityName\": {\n"
    "      \"attributes\": [\"x\", \"y\", \"z\"],\n"
    "      \"pk\": [\"a\"],\n"
    "      \"fk\": [\"b\", \"c\"]\n"
    "    },\n"
    "    \"AnotherEntity\": {\n"
    "      \"attributes\": [\"m\", \"n\"],\n"
    "      \"pk\": [\"m\"]\n"
    "    }\n"
    "  }\n"
    "}"
)

# =====================
# EXECUÇÃO
# =====================
def main():
    client = genai.Client(api_key=API_KEY)

    with open(IMAGE_PATH, "rb") as f:
        image_bytes = f.read()

    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=[
            {"inline_data": {"mime_type": "image/png", "data": image_bytes}},
            {"text": PROMPT}
        ]
    )

    print("\n=== Saída do Gemini ===\n")
    print(response.text)

if __name__ == "__main__":
    main()



=== Saída do Gemini ===

```json
{
  "Entities": {
    "EMPLOYEE": {
      "attributes": ["Ssn", "Fname", "Minit", "Lname", "Bdate", "Address", "Salary", "Sex", "Department_Number", "Start_date", "Supervisor_Ssn"],
      "pk": ["Ssn"],
      "fk": ["Department_Number", "Supervisor_Ssn"]
    },
    "DEPARTMENT": {
      "attributes": ["Number", "Name", "Manager_Ssn"],
      "pk": ["Number"],
      "fk": ["Manager_Ssn"]
    },
    "DEPARTMENT_LOCATIONS": {
      "attributes": ["Department_Number", "Location"],
      "pk": ["Department_Number", "Location"],
      "fk": ["Department_Number"]
    },
    "PROJECT": {
      "attributes": ["Number", "Name", "Location", "Controlling_Department_Number"],
      "pk": ["Number"],
      "fk": ["Controlling_Department_Number"]
    },
    "WORKS_ON": {
      "attributes": ["Employee_Ssn", "Project_Number", "Hours"],
      "pk": ["Employee_Ssn", "Project_Number"],
      "fk": ["Employee_Ssn", "Project_Number"]
    },
    "DEPENDENT": {
      "attribu